In [1]:
from dotenv import load_dotenv
from portia.cli import CLIExecutionHooks
from portia import *
import requests
import xml.etree.ElementTree as ET
from pydantic import BaseModel, Field
from typing import Generic, TypeVar, List, Literal
import os
from notion_client import Client
from my_custom_tools.registry import custom_tool_registry
import shutil

load_dotenv(override=True)

# Fetch the Notion API key and set up client
notion_api_key = os.getenv("NOTION_API_KEY")
notion_parent_id = os.getenv("NOTION_PARENT_ID")

# Initialize the Notion client
notion = Client(auth=notion_api_key)

In [2]:
my_config = Config.from_default()
complete_tool_registry = PortiaToolRegistry(my_config) + custom_tool_registry

portia = Portia(config = my_config,
                tools = complete_tool_registry,
                execution_hooks=CLIExecutionHooks(),)

In [3]:
constraints = []
# topic = input("What topic are you interested in covering today?")
# number_of_papers = int(input("How many papers do you want to download?"))
topic = "Using LLMs for Interior Design"
number_of_papers = 1
video_preference = True
number_of_videos = 3
rec_reading_preference = True
mini_project_preference = False

# Define the path to your papers folder
papers_folder = "papers"

# Ensure the papers folder exists
if not os.path.exists(papers_folder):
    os.makedirs(papers_folder)
else:
    # If it exists, clear its contents
    for filename in os.listdir(papers_folder):
        file_path = os.path.join(papers_folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # remove file or symlink
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # remove directory
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')


task = (
            lambda : f"""You are a research assistant running these tasks: 
                      - Find and download 1 paper on the topic of {topic} using the ArXivTool. 
                      - Run the PDFReaderTool to extract the full text from the pdfs in the local folder.
                      - From the text, extract the core mathematical and scientific concepts required 
                        to understand the paper. Focus only on generalizable topics that could be included 
                        in a learning pathway or curriculum—avoid content specific to the study's location, 
                        data, or outcomes. List only the overarching topics, with no explanations or extra text.
                      - Then use the TopicSelectorTool on these topics. 
                      - Then use the Notion Tool to create Notion pages for these topics.
                      - {video_preference * "Use the YouTubeTool to find videos on each topic."}
                      - {rec_reading_preference * "Use the RecReadTool to find resources on each topic."}
                      
                        Take into account these constraints: {constraints}
                      """

        )

# Iterate on the plan with the user until they are happy with it
with execution_context(end_user_id="learning_enthusiast",):
    plan = portia.plan(task())
    print("\nHere are the steps in the generated plan:")
    [print(step.model_dump_json(indent=2)) for step in plan.steps]
    ready_to_proceed = False
    while not ready_to_proceed:
        user_input = input("Are you happy with the plan? (y/n):\n")
        if user_input == "y":
            ready_to_proceed = True
        else:
            user_input = input("Any additional guidance for the planner?:\n")
            constraints.append(user_input)
            plan = portia.plan(task())
            print("\nHere are the updated steps in the plan:")
            [print(step.model_dump_json(indent=2)) for step in plan.steps]

    # Execute the plan
    print("\nThe plan will now be executed. Please wait...")
    run = portia.run_plan(plan)
    
    if run.state != PlanRunState.COMPLETE:
        raise Exception(
            f"Plan run failed with state {run.state}. Check logs for details."
        )

2025-04-12 18:15:41.694 | INFO | portia.portia:plan:197 - Running planning_agent for query - You are a research assistant running these tasks: 
                      - Find and download 1 paper on the topic of Using LLMs for Interior Design using the ArXivTool. 
                      - Run the PDFReaderTool to extract the full text from the pdfs in the local folder.
                      - From the text, extract the core mathematical and scientific concepts required 
                        to understand the paper. Focus only on generalizable topics that could be included 
                        in a learning pathway or curriculum—avoid content specific to the study's location, 
                        data, or outcomes. List only the overarching topics, with no explanations or extra text.
                      - Then use the TopicSelectorTool on these topics. 
                      - Then use the Notion Tool to create Notion pages for these topics.
                      - Use the You